## Using LangChain to get structured outputs


In [1]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_anthropic import ChatAnthropic
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import JsonOutputParser
import streamlit as st

In [2]:
claude_api_key = "<API KEY>"

In [ ]:
llm_model = ChatAnthropic(model="claude-3-haiku-20240307", api_key=claude_api_key)
# llm_model = ChatOllama(model="llama3.2", temperature=0)

### Method 1: Structured output using the tool-calling API under the hood


We can define a Pydantic model and the output will be returned as a Pydantic object with validation


In [4]:
from typing import Optional
from pydantic import BaseModel, Field


class Joke(BaseModel):
    """Joke to tell user."""

    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke")
    rating: int = Field(description="How funny the joke is, from 1 to 10")


structured_llm = llm_model.with_structured_output(Joke)
structured_llm.invoke("Tell me a joke about cats")

Joke(setup='Because it wanted to be the purr-cussionist!', punchline='Why did the cat join a band?', rating=8)

Defining the schema using a TypedDict parses the JSON output into a Python dict not a Pydantic object so there's no schema validation


In [5]:
from typing_extensions import Annotated, TypedDict


class JokeTD(TypedDict):
    """Joke to tell user."""

    setup: Annotated[str, ..., "The setup of the joke"]
    punchline: Annotated[str, ..., "The punchline of the joke"]
    rating: Annotated[Optional[int], ..., "How funny the joke is, from 1 to 10"]


structured_llm = llm_model.with_structured_output(JokeTD)
structured_llm.invoke("Tell me a joke about cats")

{'punchline': 'Why did the cat join a band?',
 'rating': 8,
 'setup': 'Because it wanted to be the purr-cussionist!'}

Or just extract the JSON Schema object


In [6]:
structured_llm = llm_model.with_structured_output(Joke.model_json_schema())
structured_llm.invoke("Tell me a joke about cats")

{'punchline': 'Why did the cat join a band?',
 'rating': 8,
 'setup': 'Because it wanted to be the purr-cussionist!'}

Let's try a more complicated structure with nested types


In [7]:
class ArticleResponse(BaseModel):
    """A clear and concise answer to the users question."""

    title: str = Field(description="Title of the article")
    context: str = Field(
        description="Provide a brief historical context to answer the question."
    )
    historical_timeline: list[str] = Field(
        description="Provide a list of historical events relevant to the question"
    )


structured_llm = llm_model.with_structured_output(ArticleResponse)
structured_llm.invoke("Tell me the history of the state of Texas in America")

ArticleResponse(title='The History of Texas in America', context='Texas has a rich and diverse history that spans thousands of years, from the earliest Native American inhabitants to its current status as the second-largest state in the US.', historical_timeline=['The Caddo and Comanche tribes inhabited the region for centuries before European exploration', 'In 1528, Álvar Núñez Cabeza de Vaca became the first European to visit Texas', 'In 1690, Spanish explorer Francisco Vásquez de Coronado arrived in Texas', 'Texas declared its independence from Mexico in 1836 and became the Republic of Texas', 'The US annexed Texas in 1845 and it became the 28th state in 1845'])

By specifying `include_raw=True` we get back the full data not just the parsed Pydantic object. This is useful if there are errors. Also we can clearly see the structures output is piggy-backing off the tool calling itnerface.


In [ ]:
structured_llm = llm_model.with_structured_output(ArticleResponse, include_raw=True)
results = structured_llm.invoke("Tell me the history of the state of Texas in America")

# Get data from tool call argyments
raw_output = results["raw"].tool_calls[0]["args"]

try:
    print(ArticleResponse(**raw_output))
except Exception as e:
    print(f"{type(e).__name__}: {str(e)}")
    print(f"\nRaw output:\n{raw_output}")

In [26]:
structured_llm = llm_model.with_structured_output(ArticleResponse, include_raw=True)
results = structured_llm.invoke("Tell me the history of the state of Texas in America")

We can directly create the JSON schema object from the Pydantic object and we get the raw dict output without Pydantic validation


In [27]:
structured_llm_js = llm_model.with_structured_output(
    ArticleResponse.model_json_schema()
)
structured_llm_js.invoke("Tell me the history of the state of Texas in America")

{'context': 'Texas has a rich and diverse history that spans thousands of years, from the earliest Native American inhabitants to its current status as the second-largest state in the US.',
 'historical_timeline': ['The Caddo and Comanche tribes inhabited the region for centuries before European exploration',
  'In 1528, Álvar Núñez Cabeza de Vaca became the first European to visit Texas',
  'In 1690, Spanish explorer Francisco Vásquez de Coronado arrived in Texas',
  'Texas declared its independence from Mexico in 1836 and became the Republic of Texas',
  'The US annexed Texas in 1845 and it became a state in 1845'],
 'title': 'The History of Texas in America'}

#### Under the hood: How Pydantic models are converted to JSONSchema


The JSON schema representation is quite straightforward


In [28]:
Joke.model_json_schema()

{'description': 'Joke to tell user.',
 'properties': {'setup': {'description': 'The setup of the joke',
   'title': 'Setup',
   'type': 'string'},
  'punchline': {'description': 'The punchline to the joke',
   'title': 'Punchline',
   'type': 'string'},
  'rating': {'description': 'How funny the joke is, from 1 to 10',
   'title': 'Rating',
   'type': 'integer'}},
 'required': ['setup', 'punchline', 'rating'],
 'title': 'Joke',
 'type': 'object'}

Note the same schema is contained in the format instructions, expect for 'title' and 'type'


In [29]:
from langchain_core.output_parsers import PydanticOutputParser

output_parser = PydanticOutputParser(pydantic_object=Joke)
print(output_parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"description": "Joke to tell user.", "properties": {"setup": {"description": "The setup of the joke", "title": "Setup", "type": "string"}, "punchline": {"description": "The punchline to the joke", "title": "Punchline", "type": "string"}, "rating": {"description": "How funny the joke is, from 1 to 10", "title": "Rating", "type": "integer"}}, "required": ["setup", "punchline", "rating"]}
```


### Method 2: JSON formating instructions


Using the PydanticOutputParser allows us to specify JSON outputs for other models that don't support tool calling.


In [30]:
from langchain_core.output_parsers import PydanticOutputParser, StrOutputParser
from langchain_core.exceptions import OutputParserException

parser = PydanticOutputParser(pydantic_object=Joke)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer the user query. Wrap the output in `json` tags\n{format_instructions}",
        ),
        ("human", "{query}"),
    ]
).partial(format_instructions=parser.get_format_instructions())

chain_llm = prompt | llm_model | parser
chain_llm.invoke("Tell me a joke about cats")

Joke(setup='Why did the cat join a band?', punchline='Because it wanted to be the purr-cussionist!', rating=8)

In [31]:
prompt_user_format = ChatPromptTemplate.from_template(
    "{input} \n{format_instructions}"
).partial(format_instructions=parser.get_format_instructions())

structured_llm = prompt_user_format | llm_model | StrOutputParser()
print(structured_llm.invoke("Tell me a joke about cats"))

{"description": "Why did the cat join a band?", "properties": {"setup": {"description": "The setup of the joke", "title": "Setup", "type": "string"}, "punchline": {"description": "The punchline to the joke", "title": "Punchline", "type": "string"}, "rating": {"description": "How funny the joke is, from 1 to 10", "title": "Rating", "type": "integer"}}, "required": ["setup", "punchline", "rating"]}

{
  "setup": "Because it wanted to be the purr-cussionist.",
  "punchline": "It was a cat-astrophe!",
  "rating": 8
}


#### Structure output with Pydantic validation


In [32]:
chain = prompt_user_format | llm_model.with_structured_output(schema=Joke)

try:
    output = chain_llm.invoke("Tell me a joke about cats")
    print(output)
except Exception as e:
    print(f"{type(e).__name__}: {str(e)}")

setup='Why did the cat join a band?' punchline='Because it wanted to be the purr-cussionist!' rating=8


#### Structured output without Pydantic validation


In [33]:
chain_llm_novalid = prompt_user_format | llm_model.with_structured_output(
    schema=Joke.model_json_schema()
)
output = chain_llm_novalid.invoke("Tell me a joke about cats")
print(type(output))
output

<class 'dict'>


{'punchline': 'Because it wanted to be the purr-cussionist.',
 'rating': 8,
 'setup': 'Why did the cat join a band?'}

#### Structured output using output parsers


Using a system prompt seems much less reliable than just inserting the format instructions into a user prompt. Why is this?


In [36]:
from langchain_core.output_parsers import PydanticOutputParser, JsonOutputParser
from langchain.output_parsers.fix import OutputFixingParser

parser = JsonOutputParser(pydantic_object=Joke)
prompt = prompt_user_format.partial(
    format_instructions=parser.get_format_instructions()
)

structured_llm = prompt | llm_model | parser

try:
    output = chain_llm_novalid.invoke("Tell me a joke about donkeys")
    print(output)

except Exception as e:
    print(f"{type(e).__name__}: {str(e)}")

{'punchline': 'Because he was caught horsing around!', 'rating': 8, 'setup': 'Why did the donkey get kicked out of the movie theater?'}


In [37]:
parser = JsonOutputParser(pydantic_object=Joke)
prompt = prompt_user_format.partial(
    format_instructions=parser.get_format_instructions()
)

structured_llm = prompt | llm_model | parser
try:
    output = chain_llm_novalid.invoke("Tell me a joke about aardvarks")
    print(output)
except Exception as e:
    print(f"{type(e).__name__}: {str(e)}")

{'punchline': 'Because he was an ear-resistible dancer!', 'rating': 8, 'setup': 'Why did the aardvark go to the party?'}


Fixing the output with `OutputFixingParser`, it could be better to use another model with lower temperature instead of the original model?


In [38]:
parser = PydanticOutputParser(pydantic_object=Joke)
prompt = prompt_user_format.partial(
    format_instructions=parser.get_format_instructions()
)
llm_model_fix = ChatOllama(model="llama3.2", temperature=0)

parser_fix = OutputFixingParser.from_llm(parser=parser, llm=llm_model_fix)

try:
    structured_llm = prompt | llm_model | parser_fix
    output = chain_llm_novalid.invoke("Tell me a joke about weevils")
    print(output)
except Exception as e:
    print(f"{type(e).__name__}: {str(e)}")

{'punchline': 'Because it was a shell of a good time!', 'rating': 8, 'setup': 'Why did the weevil go to the party?'}


### Which models support what?


In [43]:
llm_models = {
    # "Anthropic_Haiku": ChatAnthropic(model="claude-3-haiku-20240307", api_key=claude_api_key),
    "Ollama_llama32": ChatOllama(model="llama3.2", temperature=0),
    "Ollama_llama32_json": ChatOllama(model="llama3.2", format="json", temperature=0),
    "Ollama_gemma2": ChatOllama(model="gemma2", temperature=0),
    "Ollama_gemma2_json": ChatOllama(model="gemma2", format="json", temperature=0),
    "Ollama_phi3": ChatOllama(model="phi3", temperature=0),
    "Ollama_phi3_json": ChatOllama(model="phi3", format="json", temperature=0),
}

In [44]:
for llm_model in llm_models.values():
    print(f"Model: {llm_model.__repr__()}")
    test_structured_llm = llm_model.with_structured_output(JokeTD)

    try:
        output = test_structured_llm.invoke("Tell me a joke about cats")
        print("  Tool use support")
    except Exception as e:
        print(e)
        print("  No tool use")

Model: ChatOllama(model='llama3.2', temperature=0.0)
  Tool use support
Model: ChatOllama(model='llama3.2', temperature=0.0, format='json')
  Tool use support
Model: ChatOllama(model='gemma2', temperature=0.0)
registry.ollama.ai/library/gemma2:latest does not support tools (status code: 400)
  No tool use
Model: ChatOllama(model='gemma2', temperature=0.0, format='json')
registry.ollama.ai/library/gemma2:latest does not support tools (status code: 400)
  No tool use
Model: ChatOllama(model='phi3', temperature=0.0)
registry.ollama.ai/library/phi3:latest does not support tools (status code: 400)
  No tool use
Model: ChatOllama(model='phi3', temperature=0.0, format='json')
registry.ollama.ai/library/phi3:latest does not support tools (status code: 400)
  No tool use
